In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#### ~~В качестве dataset’а берем Iris, оставив 2 класса: Iris Versicolor, Iris Virginica.~~ Исходный набор данных, указанный в условии задачи, к сожалению не давал мне ответа хорошо ли моя логистическая регрессия работает. Моя регрессия, как и sklearn.linear_model.LogisticRegression, после обучения всегда классифицировали одним и тем же классом, давая одинаковое качество 0.6. Мой результат совпадал с решением из коробки, но это ровным счетом ничего не значило. Данные не подходили для линейной модели. Поэтому я модифицировал исходный набор данных, оставив те значения выходных параметров и те искомые значения, которые подходят для работы с линейной моделью.

In [79]:
data = load_iris()
X = data.data[data.target != 2, :2]
y = data.target[data.target != 1]

#### Т.к. остались классы 1 и 2, преобразую их в значений 0 и 1.

In [80]:
y[y > 0] = 1

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [75]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

#### Производная от Loss-функции.
~~Функция имеет следующий вид:
$$J = -\sum_{i=1}^{N} y_i\log (h_\theta(x_i)) + (1 - y_i)\log(1 - h_\theta(x_i))$$
Возьмем от нее производную:
$$(y_i\log (h_\theta(x_i)) + (1 - y_i)\log(1 - h_\theta(x_i)))'$$
Производная от суммы равна сумме производных:
$$((y_i\log (h_\theta(x_i)))' + ((1 - y_i)\log(1 - h_\theta(x_i)))'$$
Раскрываем первую скобку, считая производную от произведения по формуле (x*y)' = x*y' + x'*y:
$$y_i(log (h_\theta(x_i))' + (y_i)'\log (h_\theta(x_i) + ((1 - y_i)\log(1 - h_\theta(x_i)))'$$
Производная от y равна 1, производная от ln(x) равна $\frac{1}{x}$:
$$\frac{y_i}{h_\theta(x_i)} + log (h_\theta(x_i)) + ((1 - y_i)\log(1 - h_\theta(x_i)))'$$
Раскрываем вторую скобку ровно по тем же правилам:
$$\frac{y_i}{h_\theta(x_i)} + log (h_\theta(x_i)) + (1 - y_i)\log(1 - h_\theta(x_i))' + (1 - y_i)'\log(1 - h_\theta(x_i))$$
$$\frac{y_i}{h_\theta(x_i)} + log (h_\theta(x_i)) + \frac{1 - y_i}{1 - h_\theta(x_i)} + (1' - y_i')\log(1 - h_\theta(x_i))$$
Производная от 1 равна нулю:
$$\frac{y_i}{h_\theta(x_i)} + log (h_\theta(x_i)) + \frac{1 - y_i}{1 - h_\theta(x_i)} - \log(1 - h_\theta(x_i))$$
Переупорядочиваем:
$$\frac{y_i}{h_\theta(x_i)} + \frac{1 - y_i}{1 - h_\theta(x_i)} + log (h_\theta(x_i)) - \log(1 - h_\theta(x_i))$$
Все это суммируем и берем со знаком минус.~~

#### На самом деле все не так!
Я понял, что я неправильно дифференцировал, но что именно я сделал не так, понять у меня не получилось. Судя по всему я ошибся когда брал производную от произведения и от логарифма. Производная от функции sigmoid все же была нужна, после раскрытия логарифма. Так или иначе градиент чудесным образом оказался равен градиенту линейной регрессии и это значительно облегчает задачу! :)

### Метод градиентного спуска

In [136]:
class GradientClassifier:
    
    params = []
    
    def __init__(self, learning_rate = 0.01, epochs = 100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.params = []
    
    def sigmoid(self, y_hat):
        return 1 / (1 + np.exp(-y_hat))
    
    def fit(self, X, y):
        
        self.params = np.random.normal(size=(3,))
        for _ in range(self.epochs):
            y_hat = self.params[0] + self.params[1] * X[:, 0] + self.params[2] * X[:, 1] #+ self.params[3] * X[:, 2] + self.params[4] * X[:, 3]
            y_pred = self.sigmoid(y_hat)

            self.params[0] -= self.learning_rate * np.sum(y_pred - y) / len(y_pred)
            self.params[1] -= self.learning_rate * np.sum((y_pred - y) * X[:, 0]) / len(y_pred)
            self.params[2] -= self.learning_rate * np.sum((y_pred - y) * X[:, 1]) / len(y_pred)
#            self.params[3] -= self.learning_rate * np.sum((y_pred - y) * X[:, 2]) / len(y_pred)
#            self.params[4] -= self.learning_rate * np.sum((y_pred - y) * X[:, 3]) / len(y_pred)

    def intercept_(self):
        return [params[0]]
    
    def coef_(self):
        return [params[1:]]

    def predict(self, X):
        y_hat = self.params[0] + self.params[1] * X[:, 0] + self.params[2] * X[:, 1] #+ self.params[3] * X[:, 2] + self.params[4] * X[:, 3]
        y_pred = self.sigmoid(y_hat)
        return (y_pred > 0.5) * 1
        
    def score(self, X, y):
        y_predict = self.predict(X)
        return np.sum(y_predict == y) / len(y)

In [137]:
cls = GradientClassifier()
cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)
print('{} {}'.format([cls.params[0]], cls.params[1:]))
print(y_pred)
print(y_test)
print(cls.score(X_test, y_test))

[-0.4551371089134761] [ 1.07654745 -1.71436838]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0]
0.9666666666666667


In [102]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('{} {}'.format(lr.intercept_, lr.coef_))
print(y_pred)
print(y_test)
print(lr.score(X_test, y_test))

[-0.30257083] [[ 1.95425259 -3.29682321]]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0]
0.9666666666666667


/home/myxrome/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Мои коэффициенты отличаются от коэффициентов sklearn.linear_model.LogisticRegression, но имеют одинаковое направление. Возможно, нужно поменять гиперпараметры обучения: шаг обучения и количество итераций. Возможно решение sklearn использует более продвинутый алгоритм, а мое решение не в состоянии найти тот самый минимум и ходит очень рядом с ним. Тем не менее, результаты предсказаний одинаковы. 

### Nesterov momentum

In [157]:
class NesterovClassifier:
    
    params = []
    
    def __init__(self, ips = 0.975, learning_rate = 0.025, epochs = 100):
        self.ips = ips
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.params = []
    
    def sigmoid(self, y_hat):
        return 1 / (1 + np.exp(-y_hat))
    
    def fit(self, X, y):
        
        self.params = np.random.normal(size=(3,))
        prev_delta = np.zeros(shape=(3,))
        for _ in range(self.epochs):
            new_params = self.params - prev_delta
            y_hat = new_params[0] + new_params[1] * X[:, 0] + new_params[2] * X[:, 1] #+ new_params[3] * X[:, 2] + new_params[4] * X[:, 3]
            y_pred = self.sigmoid(y_hat)
            
            grad = np.zeros(shape=(3,))
            grad[0] = np.sum(y_pred - y) / len(y_pred)
            grad[1] = np.sum((y_pred - y) * X[:, 0]) / len(y_pred)
            grad[2] = np.sum((y_pred - y) * X[:, 1]) / len(y_pred)

            prev_delta[0] = self.ips * prev_delta[0] + self.learning_rate * grad[0]
            prev_delta[1] = self.ips * prev_delta[1] + self.learning_rate * grad[1]
            prev_delta[2] = self.ips * prev_delta[2] + self.learning_rate * grad[2]
            
            self.params -= prev_delta

    def intercept_(self):
        return [params[0]]
    
    def coef_(self):
        return [params[1:]]

    def predict(self, X):
        y_hat = self.params[0] + self.params[1] * X[:, 0] + self.params[2] * X[:, 1] #+ self.params[3] * X[:, 2] + self.params[4] * X[:, 3]
        y_pred = self.sigmoid(y_hat)
        return (y_pred > 0.5) * 1
        
    def score(self, X, y):
        y_predict = self.predict(X)
        return np.sum(y_predict == y) / len(y)

In [167]:
cls = NesterovClassifier()
cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)
print('{} {}'.format([cls.params[0]], cls.params[1:]))
print(y_pred)
print(y_test)
print(cls.score(X_test, y_test))

[-1.29186380228296] [ 3.94710537 -6.42473769]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0]
0.9666666666666667


#### Прогон градиентного спуска очень сильно зависит от начального случайного значения и конечное качество может колебаться от 0.1 до 0.96.  Метод Nesterov показывает более стабильное обучение. Качество ниже 0.9 за 10 прогонов у меня ни разу не опустилось.

### RMSProp

In [168]:
import math

In [196]:
class RMSPropClassifier:
    
    params = []
    
    def __init__(self, ips = 0.5, sigma = 0.0001, learning_rate = 0.1, epochs = 100):
        self.ips = ips
        self.sigma = sigma
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.params = []
    
    def sigmoid(self, y_hat):
        return 1 / (1 + np.exp(-y_hat))
    
    def fit(self, X, y):
        
        self.params = np.random.normal(size=(3,))
        G = np.zeros(shape=(3,))
        for _ in range(self.epochs):
            y_hat = self.params[0] + self.params[1] * X[:, 0] + self.params[2] * X[:, 1] #+ self.params[3] * X[:, 2] + self.params[4] * X[:, 3]
            y_pred = self.sigmoid(y_hat)
            
            grad = np.zeros(shape=(3,))
            grad[0] = np.sum(y_pred - y) / len(y_pred)
            grad[1] = np.sum((y_pred - y) * X[:, 0]) / len(y_pred)
            grad[2] = np.sum((y_pred - y) * X[:, 1]) / len(y_pred)

            G[0] = self.ips * G[0] + (1 - self.ips) * math.pow(grad[0], 2)
            G[1] = self.ips * G[1] + (1 - self.ips) * math.pow(grad[1], 2)
            G[2] = self.ips * G[2] + (1 - self.ips) * math.pow(grad[2], 2)
            
            self.params[0] -= (self.learning_rate / math.sqrt(G[0] + self.sigma)) * grad[0]
            self.params[1] -= (self.learning_rate / math.sqrt(G[1] + self.sigma)) * grad[1]
            self.params[2] -= (self.learning_rate / math.sqrt(G[2] + self.sigma)) * grad[2]
            
    def intercept_(self):
        return [params[0]]
    
    def coef_(self):
        return [params[1:]]

    def predict(self, X):
        y_hat = self.params[0] + self.params[1] * X[:, 0] + self.params[2] * X[:, 1] #+ self.params[3] * X[:, 2] + self.params[4] * X[:, 3]
        y_pred = self.sigmoid(y_hat)
        return (y_pred > 0.5) * 1
        
    def score(self, X, y):
        y_predict = self.predict(X)
        return np.sum(y_predict == y) / len(y)

In [207]:
cls = RMSPropClassifier()
cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)
print('{} {}'.format([cls.params[0]], cls.params[1:]))
print(y_pred)
print(y_test)
print(cls.score(X_test, y_test))

[-1.7521530844321347] [ 2.0852589  -2.92359122]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1]
[1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0]
0.9666666666666667


#### Данный метод очень сильно зависит от входных параметров. Мне удалось подобрать параметры так, чтобы итоговое качество не опускалось ниже 0.9 от обучения к обучению.

#### P.S.: Операции над множествами через элементы с дублированием строчек не очень элегантное решение, но я так делал для наглядности и решил эту наглядность оставить в конечном варианте.